In [0]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal

In [0]:
#preprocessing data
with open('multigauss.txt') as f:
  data = f.readlines()  
  data_matrix = [line.strip().split(" ") for line in data[4:]]
  f.close()
data_matrix = np.array([[float(x) for x in line] for line in data_matrix])

In [3]:
data_matrix

array([[ 1.34397344,  0.15110434],
       [ 0.96267177,  0.38278123],
       [ 0.50232576, -0.1813927 ],
       ...,
       [-3.99595629,  3.21112544],
       [-3.0057663 ,  1.7203983 ],
       [-3.29047557,  0.73752212]])

In [0]:
# number of training points
n = data_matrix.shape[0]
# number of features
f = data_matrix.shape[1]
# number of gaussians
k = 5
# training data
X = data_matrix
# membership weights
w = np.random.rand(n, k)
# gaussian prior probabilities
_pi = np.random.rand(k, 1)
# print(_pi)
# cluster means
_mu = np.random.rand(k, f)
# covariance matrics
_sigma = np.random.rand(f, f, k)


In [0]:
#convert matrix to its positive semi definate
def getPosSemiDefinite(m):
  for i in range(k):
    m[:,:,i] = np.dot(m[:,:,i], m[:,:,i].T)
  return m

In [0]:
_sigma = getPosSemiDefinite(_sigma)

In [0]:
#E-step
def Expectation(X, k, _pi, _mu, _sigma):
  
  total_multi_gauss = np.zeros((X.shape[0],))
  
  for i in range(k):
    total_multi_gauss += (_pi[i]) * multivariate_normal(_mu[i, :], _sigma[:,:,i]).pdf(X)   
  
  for c in range(k):    
    numer = _pi[c] * multivariate_normal(_mu[c, :], _sigma[:,:,c]).pdf(X)
    denom = total_multi_gauss    
    w[:, c] = numer / denom  
    
  return w
  
    
# Expectation(X, k, _pi, _mu, _sigma)   


In [0]:
def MaximizeMean(X, k, w):
  
  mu = np.random.rand(k, X.shape[1])
  
  for c in range(k):
    total_w = np.sum(w[:,c])
    
    mu[c] = (1/total_w) * np.dot(w[:,c], X)
    
  return mu
  
# MaximizeMean(X, k, w)

In [0]:
def MaximizeCovariance(X, k, w, _mu):
  
  sigma = np.zeros((f,f,k))
  
  
  for c in range(k):
    
    total_w = np.sum(w[:,c])  
    
    sigma_i = np.zeros((X.shape[1], X.shape[1])) 
    
    for i in range(X.shape[0]):
      sigma_i += w[i,c] * ((X[i, :] - _mu[c,:]).reshape(2,1).T * (X[i, :] - _mu[c,:]).reshape(2,1))
    
    sigma[:,:,c] = ((1 / total_w) * sigma_i) 
    
  return sigma

# MaximizeCovariance(X, k, w, _mu)


In [0]:
def loglikelihood(X, k, _pi, _mu, _sigma):
  
  l = 0
  
  for i in range(X.shape[0]):
    
    total_ = 0
  
    for c in range(k):
      total_ += (_pi[c]) * multivariate_normal(_mu[c, :], _sigma[:,:,c]).pdf(X[i, :]) 
    
    l += np.log(total_)
  
  return l

# loglikelihood(X, k, _pi, _mu, _sigma)[0]

In [0]:
def MaximizeMixtures(k, w):
  
  pi = np.random.rand(k, 1)
  
  total_w = np.sum(w)
    
  for c in range(k):
    
    total_w_per_cluster = np.sum(w[:, c])
    
    pi[c] = total_w_per_cluster / total_w
    
  return pi


# MaximizeMixtures(k ,w)
    

In [0]:
def EM(X, k, _pi, _mu, _sigma, nIter):  
  
  for i in range(nIter):
      
    w = Expectation(X, k, _pi, _mu, _sigma)
    _pi = MaximizeMixtures(k, w)
    _mu = MaximizeMean(X, k, w)
    _sigma = MaximizeCovariance(X, k, w, _mu)
    
    ll = loglikelihood(X, k, _pi, _mu, _sigma)[0]
    print('Iteration %d: log-likelihood is %.4f' % (i, ll) ) 
  
  return _pi,_mu,_sigma

In [14]:
# here niter=50, this can be reduced to terminate program earlier!
EM(X, k, _pi, _mu, _sigma, 50)


Iteration 0: log-likelihood is -6122.1531
Iteration 1: log-likelihood is -6091.1262
Iteration 2: log-likelihood is -6072.9547
Iteration 3: log-likelihood is -6055.7019
Iteration 4: log-likelihood is -6037.2908
Iteration 5: log-likelihood is -6010.8253
Iteration 6: log-likelihood is -5970.3065
Iteration 7: log-likelihood is -5921.6706
Iteration 8: log-likelihood is -5875.0802
Iteration 9: log-likelihood is -5831.5712
Iteration 10: log-likelihood is -5787.4571
Iteration 11: log-likelihood is -5742.4820
Iteration 12: log-likelihood is -5700.5855
Iteration 13: log-likelihood is -5662.2820
Iteration 14: log-likelihood is -5634.9565
Iteration 15: log-likelihood is -5618.1546
Iteration 16: log-likelihood is -5608.3065
Iteration 17: log-likelihood is -5601.5784
Iteration 18: log-likelihood is -5596.5350
Iteration 19: log-likelihood is -5592.8300
Iteration 20: log-likelihood is -5590.3074
Iteration 21: log-likelihood is -5588.7273
Iteration 22: log-likelihood is -5587.7713
Iteration 23: log-lik

(array([[0.11393141],
        [0.08841957],
        [0.59649165],
        [0.19279881],
        [0.00835856]]), array([[-3.59328884, -2.89412049],
        [-2.52194478, -3.12403604],
        [-0.11529306, -0.00490964],
        [ 2.90574997,  3.05267189],
        [ 0.61177165,  0.2632155 ]]), array([[[ 0.69778146,  0.52944496,  6.97989947,  0.84555673,
           0.27790253],
         [ 0.10337431,  0.06083227, -6.03488331, -0.06345865,
           0.1010081 ]],
 
        [[ 0.10337431,  0.06083227, -6.03488331, -0.06345865,
           0.1010081 ],
         [ 1.01683665,  0.96704092,  7.03430667,  0.84921123,
           0.03774358]]]))